In [1]:
from smart_open import open
import pandas as pd
import datetime
import numpy as np

In [2]:
data = pd.read_csv("../data/training_pruned_file_20162021.csv")

In [3]:
data = data.drop([
    "continent", 
    "player_id_transfermt", 
    "date", 
    "position_id_y", 
    "country_name", 
    "gender", 
    "common_name", 
    "firstname_x", 
    "lastname_x", 
    "name", 
    "height_x", 
    "weight_x", 
    "league.season", 
    "agent_name", 
    "global_performance",
    "birth.country",
    "nationality",
], axis=1)

In [4]:
data["age"] = pd.to_datetime(data["date_of_birth"]).map(lambda x: int((pd.Timestamp.today() - x).days / 365))
del data["date_of_birth"]

In [5]:
data[lambda x: x["player_id_api"] == 154]

,player_id_api,team_name,season_name,league_name,accurate_crosses.total,accurate_passes.total,aerials_won.total,bench.total,big_chances_created.total,big_chances_missed.total,...,penalty.missed,penalty.saved,position,sub_position,foot,market_value_in_eur_x,highest_market_value_in_eur,position_name_x,position_name_y,age
2083,154,FC Barcelona,2017,La Liga,NaN,NaN,NaN,4.0,NaN,NaN,...,2.0,NaN,Attack,Right Winger,Left,180000000,180000000.0,Attacker,Right Wing,35
5254,154,FC Barcelona,2018,La Liga,12.0,1168.0,NaN,5.0,NaN,NaN,...,1.0,NaN,Attack,Right Winger,Left,150000000,180000000.0,Attacker,Right Wing,35
7466,154,FC Barcelona,2019,La Liga,27.0,1349.0,NaN,1.0,NaN,NaN,...,0.0,NaN,Attack,Right Winger,Left,112000000,180000000.0,Attacker,Right Wing,35
10914,154,FC Barcelona,2020,La Liga,35.0,1914.0,8.0,2.0,NaN,NaN,...,2.0,NaN,Attack,Right Winger,Left,80000000,180000000.0,Attacker,Right Wing,35
13566,154,Paris Saint Germain,2021,Ligue 1,8.0,1390.0,5.0,2.0,17.0,11.0,...,0.0,NaN,Attack,Right Winger,Left,50000000,180000000.0,Attacker,Right Wing,35


In [192]:
before_after_seasons = pd.merge(data, data, on="player_id_api", suffixes=["_before", "_after"]).query("season_name_after - season_name_before == 1")

for c in [
    "foot_after",
    "age_before",
    "season_name_after",
    "season_name_before",
]:
    del before_after_seasons[c]
    
for c in [
    "games.position",
    "highest_market_value_in_eur",
    "league.country",
    "league_name",
    "position",
    "position_name_x",
    "position_name_y",
    "sub_position",
    "team_name",
]:
    before_after_seasons[f"same_{c}"] = before_after_seasons[f"{c}_before"] == before_after_seasons[f"{c}_after"]
    del before_after_seasons[f"{c}_before"]
    del before_after_seasons[f"{c}_after"]


In [193]:
before_after_seasons = before_after_seasons[[d[0] for d in before_after_seasons.dtypes.items() if d[1] != object]]

In [194]:
def label(mv_diff):
    if mv_diff > 0:
        return "inc"
    elif mv_diff < 0:
        return "dec"
    else:
        return "same"
before_after_seasons["label"] = (before_after_seasons["market_value_in_eur_x_after"] - before_after_seasons["market_value_in_eur_x_before"]).map(label)
del before_after_seasons["market_value_in_eur_x_after"]
del before_after_seasons["market_value_in_eur_x_before"]

In [195]:
columns = {
    c
    for c in before_after_seasons.columns
    if "same_" not in c
}
columns.remove("label")

In [196]:
for c in columns:
    before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
before_after_seasons = before_after_seasons.copy()

/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  before_after_seasons[f"has_{c}"] = ~before_after_seasons[c].isnull()
/var/folders/h8/j63w626x3ds50k3yc8j6xtwh0000gn/T/ipykernel_75782/2635444764.py:2: PerformanceWarning: DataFrame is highly fragmented.  This 

In [197]:
before_after_seasons

,player_id_api,accurate_crosses.total_before,accurate_passes.total_before,aerials_won.total_before,bench.total_before,big_chances_created.total_before,big_chances_missed.total_before,blocked_shots.total_before,captain.total_before,clearances.total_before,...,has_fouls.total_before,has_redcards.away_before,has_accurate_passes.total_after,has_redcards.home_before,has_dribbled_past.total_before,has_goals.conceded_before,has_goals.goals_before,has_yellowred_cards.home_before,has_yellowcards.away_before,has_blocked_shots.total_before
2,18778,NaN,6.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,...,True,False,True,False,False,False,False,False,False,False
8,18778,3.0,167.0,NaN,6.0,NaN,NaN,NaN,NaN,4.0,...,True,False,True,False,True,False,True,False,False,False
14,18778,5.0,416.0,NaN,14.0,NaN,NaN,NaN,NaN,3.0,...,True,False,True,False,True,False,True,False,False,False
20,18778,3.0,455.0,24.0,6.0,NaN,NaN,NaN,NaN,9.0,...,True,False,True,False,True,True,True,False,False,False
28,18777,38.0,446.0,NaN,5.0,NaN,NaN,NaN,NaN,25.0,...,True,True,True,True,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54009,286559,NaN,126.0,11.0,NaN,NaN,NaN,NaN,NaN,24.0,...,True,False,True,False,True,True,False,False,True,False
54016,122233,NaN,112.0,42.0,12.0,NaN,NaN,NaN,NaN,10.0,...,True,False,True,False,True,True,True,False,True,False
54025,47120,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,False
54038,119742,3.0,67.0,8.0,10.0,NaN,NaN,NaN,NaN,10.0,...,True,False,True,False,True,True,False,False,True,False


In [198]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(before_after_seasons.iloc[2].sort_index())

accurate_crosses.total_after                 3.0
accurate_crosses.total_before                5.0
accurate_passes.total_after                455.0
accurate_passes.total_before               416.0
aerials_won.total_after                     24.0
aerials_won.total_before                     NaN
age_after                                     25
bench.total_after                            6.0
bench.total_before                          14.0
big_chances_created.total_after              NaN
big_chances_created.total_before             NaN
big_chances_missed.total_after               NaN
big_chances_missed.total_before              NaN
blocked_shots.total_after                    NaN
blocked_shots.total_before                   NaN
captain.total_after                          NaN
captain.total_before                         NaN
cards.red_after                              0.0
cards.red_before                             0.0
cards.yellow_after                           0.0
cards.yellow_before 

In [199]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [375]:
y = before_after_seasons["label"]
# X = before_after_seasons.drop(["label", "player_id_api"], axis=1).fillna(0)
X = before_after_seasons[[
    # "player_id_api",
    "age_after",
    # "has_games.minutes_before",
    # "has_games.minutes_after",
    "games.minutes_before",
    "games.minutes_after",
    # "has_minutes_played.total_before",
    # "has_minutes_played.total_after",
    # "minutes_played.total_before",
    # "minutes_played.total_after",
    # "goals.goals_before",
    # "goals.goals_after",
    # "total_duels.total_before",
    # "total_duels.total_after",
    # "bench.total_before",
    # "bench.total_after",
]].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [378]:
len(X)

8910

In [355]:
(y_train == "dec").sum() / len(y_train)

0.5763187429854096

In [357]:
(y_train == "inc").sum() / len(y_train)

0.30667789001122336

In [356]:
(y_train == "same").sum() / len(y_train)

0.117003367003367

In [231]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.dummy import DummyClassifier

In [376]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('lr', LogisticRegression(random_state=42))
    # ('svc', SVC())
    ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
print(
    "random",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test, y_test),
    sep="\n"
)

random
confusion_matrix
[[179  69 335]
 [ 61  31 112]
 [332  93 570]]
balanced_accuracy_score
0.34395256532440793
accuracy_score
0.4377104377104377
pipe.score
0.4382716049382716


In [377]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42))
    # ('svc', SVC())
    # ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
print(
    "logistic regression",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test, y_test),
    sep="\n"
)

logistic regression
confusion_matrix
[[338   0 245]
 [ 48   0 156]
 [122   0 873]]
balanced_accuracy_score
0.4857155991506992
accuracy_score
0.6795735129068462
pipe.score
0.6795735129068462


In [348]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    # ('lr', LogisticRegression(random_state=42))
    ('svc', SVC())
    # ("random", DummyClassifier(strategy="stratified"))
])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
print(
    "svc",
    "confusion_matrix", 
    confusion_matrix(y_test, y_pred, labels=["inc","same","dec"]), 
    "balanced_accuracy_score",
    balanced_accuracy_score(y_test, y_pred),
    "accuracy_score",
    accuracy_score(y_test, y_pred),
    "pipe.score",
    pipe.score(X_test, y_test),
    sep="\n"
)

svc
confusion_matrix
[[393   3 187]
 [ 63  12 129]
 [ 90   7 898]]
balanced_accuracy_score
0.5451451925486918
accuracy_score
0.7312008978675645
pipe.score
0.7312008978675645
